<a href="https://colab.research.google.com/github/kridtapon/WFO-MA-Envelope/blob/main/WFO_MA_Envelope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.6 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Moving Average Envelope
def calculate_ma_envelope(df, ma_period=20, envelope_pct=0.02):
    """
    Calculate Moving Average Envelope (upper and lower bounds).
    """
    ma = df['Close'].rolling(window=ma_period).mean()  # Calculate moving average
    upper_envelope = ma * (1 + envelope_pct)  # Upper envelope (percentage above MA)
    lower_envelope = ma * (1 - envelope_pct)  # Lower envelope (percentage below MA)
    return upper_envelope, lower_envelope

# Define the stock symbol and time period
symbol = 'BTC-USD'  # SPY is the symbol for the S&P 500 ETF
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate Moving Average Envelope (using 20-period MA and 2% envelope)
df['Upper_Envelope'], df['Lower_Envelope'] = calculate_ma_envelope(df, ma_period=20, envelope_pct=0.02)

# Define Entry and Exit signals based on Moving Average Envelope strategy
df['Entry'] = df['Close'] > df['Upper_Envelope']  # Entry when price closes above the upper envelope
df['Exit'] = df['Close'] < df['Lower_Envelope']  # Exit when price closes below the lower envelope

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed


Start                                2020-01-01 00:00:00
End                                  2024-12-31 00:00:00
Period                                1827 days 00:00:00
Start Value                                     100000.0
End Value                                 1227753.625992
Total Return [%]                             1127.753626
Benchmark Return [%]                         1197.596406
Max Gross Exposure [%]                             100.0
Total Fees Paid                             43578.098884
Max Drawdown [%]                               55.286243
Max Drawdown Duration                  846 days 00:00:00
Total Trades                                          44
Total Closed Trades                                   44
Total Open Trades                                      0
Open Trade PnL                                       0.0
Win Rate [%]                                   43.181818
Best Trade [%]                                 65.174506
Worst Trade [%]                

In [42]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import itertools

# Function to calculate Moving Average Envelope (upper and lower bounds)
def calculate_ma_envelope(df, ma_period=20, envelope_pct=0.02):
    """
    Calculate Moving Average Envelope (upper and lower bounds).
    """
    ma = df['Close'].rolling(window=ma_period).mean()  # Calculate moving average
    upper_envelope = ma * (1 + envelope_pct)  # Upper envelope (percentage above MA)
    lower_envelope = ma * (1 - envelope_pct)  # Lower envelope (percentage below MA)
    return upper_envelope, lower_envelope

# Define the stock symbol and time period
symbol = 'TPR' # TPR
start_date = '2015-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Walk-forward optimization with Moving Average Envelope
def walk_forward_optimization_ma_envelope(df, start_year, end_year):
    results = []

    # Define dynamic ranges for parameters
    envelope_pct_range = np.arange(0.01, 0.05, 0.01)  # Envelope percentage range (1% to 5%)
    ma_period_range = range(5, 31)  # Range for MA periods (5 to 50)

    for test_year in range(start_year + 4, end_year + 1):
        train_start = test_year - 4
        train_end = test_year - 1
        test_start = test_year

        train_data = df[(df.index.year >= train_start) & (df.index.year <= train_end)]
        test_data = df[df.index.year == test_year]

        best_params = None
        best_performance = -np.inf

        # Loop through all combinations of MA Envelope parameters
        for params in itertools.product(ma_period_range, envelope_pct_range):
            ma_period, envelope_pct = params

            # Calculate Moving Average Envelope on the training data
            train_data['Upper_Envelope'], train_data['Lower_Envelope'] = calculate_ma_envelope(train_data, ma_period, envelope_pct)

            # Generate entry and exit signals based on MA Envelope strategy
            entries = train_data['Close'] > train_data['Upper_Envelope']  # Entry when price closes above the upper envelope
            exits = train_data['Close'] < train_data['Lower_Envelope']  # Exit when price closes below the lower envelope

            # Backtest on training data
            portfolio = vbt.Portfolio.from_signals(
                close=train_data['Close'],
                entries=entries,
                exits=exits,
                init_cash=100_000,
                fees=0.001
            )

            performance = portfolio.total_return()
            if performance > best_performance:
                best_performance = performance
                best_params = (ma_period, envelope_pct)

        # Test with the best parameters on the test data
        test_data['Upper_Envelope'], test_data['Lower_Envelope'] = calculate_ma_envelope(test_data, best_params[0], best_params[1])

        entries = test_data['Close'] > test_data['Upper_Envelope']  # Entry when price closes above the upper envelope
        exits = test_data['Close'] < test_data['Lower_Envelope']  # Exit when price closes below the lower envelope

        portfolio = vbt.Portfolio.from_signals(
            close=test_data['Close'],
            entries=entries,
            exits=exits,
            init_cash=100_000,
            fees=0.001
        )

        results.append({
            'Year': test_year,
            'Best_Params': best_params
        })

    return pd.DataFrame(results)


# Perform walk-forward optimization with MA Envelope strategy
results = walk_forward_optimization_ma_envelope(df, 2016, 2025)

# Display results
print("\nWalk-Forward Optimization Results:")
print(results)

# Combine signals into a single portfolio
combined_entries = pd.Series(False, index=df.index)
combined_exits = pd.Series(False, index=df.index)

for _, row in results.iterrows():
    year = row['Year']
    params = row['Best_Params']
    yearly_data = df[df.index.year == year]

    # Apply Moving Average Envelope strategy
    yearly_data['Upper_Envelope'], yearly_data['Lower_Envelope'] = calculate_ma_envelope(yearly_data, params[0], params[1])

    # Define entry/exit conditions
    entries = yearly_data['Close'] > yearly_data['Upper_Envelope']
    exits = yearly_data['Close'] < yearly_data['Lower_Envelope']

    combined_entries.loc[entries.index] = entries
    combined_exits.loc[exits.index] = exits

# Filter data for testing period only
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]
combined_entries = combined_entries[(combined_entries.index.year >= 2020) & (combined_entries.index.year <= 2025)]
combined_exits = combined_exits[(combined_exits.index.year >= 2020) & (combined_exits.index.year <= 2025)]

# Backtest using the combined signals
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=combined_entries,
    exits=combined_exits,
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
<ipython-input-42-85dc0154fb2b>:51: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-42-85dc0154fb2b>:51: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-42-85dc0154fb2b>:72: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide


Walk-Forward Optimization Results:
   Year Best_Params
0  2020  (27, 0.04)
1  2021   (8, 0.02)
2  2022   (8, 0.02)
3  2023   (5, 0.04)
4  2024   (5, 0.04)
5  2025   (5, 0.04)
Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           283406.164147
Total Return [%]                       183.406164
Benchmark Return [%]                   174.090919
Max Gross Exposure [%]                      100.0
Total Fees Paid                       9502.944911
Max Drawdown [%]                        46.890882
Max Drawdown Duration                       887.0
Total Trades                                   33
Total Closed Trades                            32
Total Open Trades                               1
Open Trade PnL                      110415.134987
Win Rate [%]                               34.375
Best Trade [%]          

<ipython-input-42-85dc0154fb2b>:110: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-42-85dc0154fb2b>:110: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-42-85dc0154fb2b>:110: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-42-8

In [41]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import itertools

# Function to calculate Moving Average Envelope (upper and lower bounds)
def calculate_ma_envelope(df, ma_period=20, envelope_pct=0.02):
    """
    Calculate Moving Average Envelope (upper and lower bounds).
    """
    ma = df['Close'].rolling(window=ma_period).mean()  # Calculate moving average
    upper_envelope = ma * (1 + envelope_pct)  # Upper envelope (percentage above MA)
    lower_envelope = ma * (1 - envelope_pct)  # Lower envelope (percentage below MA)
    return upper_envelope, lower_envelope

# Function to calculate adaptive envelope percentage based on volatility
def calculate_adaptive_envelope_pct(df, window=20):
    """
    Calculate envelope percentage based on market volatility (standard deviation of returns).
    """
    # Calculate daily returns
    returns = df['Close'].pct_change()

    # Calculate volatility (standard deviation of returns) over a rolling window
    volatility = returns.rolling(window=window).std()

    # Normalize the volatility to get a dynamic envelope percentage (e.g., 1-5%)
    envelope_pct = volatility * 2  # Scale the volatility to get an envelope percentage

    # Cap the envelope percentage to a reasonable range
    envelope_pct = envelope_pct.clip(lower=0.01, upper=0.05)  # Cap between 1% and 5%

    return envelope_pct

# Define the stock symbol and time period
symbol = 'TPR' # TPR DECK
start_date = '2015-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Walk-forward optimization with Moving Average Envelope
def walk_forward_optimization_ma_envelope(df, start_year, end_year):
    results = []

    # Define range for MA periods (5 to 50)
    ma_period_range = range(5, 31)

    for test_year in range(start_year + 4, end_year + 1):
        train_start = test_year - 4
        train_end = test_year - 1
        test_start = test_year

        train_data = df[(df.index.year >= train_start) & (df.index.year <= train_end)]
        test_data = df[df.index.year == test_year]

        best_params = None
        best_performance = -np.inf

        # Loop through all combinations of MA Envelope parameters
        for ma_period in ma_period_range:
            # Calculate adaptive envelope percentage based on volatility
            envelope_pct = calculate_adaptive_envelope_pct(train_data).iloc[-1]  # Use latest volatility value

            # Calculate Moving Average Envelope on the training data
            train_data['Upper_Envelope'], train_data['Lower_Envelope'] = calculate_ma_envelope(train_data, ma_period, envelope_pct)

            # Generate entry and exit signals based on MA Envelope strategy
            entries = train_data['Close'] > train_data['Upper_Envelope']  # Entry when price closes above the upper envelope
            exits = train_data['Close'] < train_data['Lower_Envelope']  # Exit when price closes below the lower envelope

            # Backtest on training data
            portfolio = vbt.Portfolio.from_signals(
                close=train_data['Close'],
                entries=entries,
                exits=exits,
                init_cash=100_000,
                fees=0.001
            )

            performance = portfolio.total_return()
            if performance > best_performance:
                best_performance = performance
                best_params = (ma_period, envelope_pct)

        # Test with the best parameters on the test data
        test_data['Upper_Envelope'], test_data['Lower_Envelope'] = calculate_ma_envelope(test_data, best_params[0], best_params[1])

        entries = test_data['Close'] > test_data['Upper_Envelope']  # Entry when price closes above the upper envelope
        exits = test_data['Close'] < test_data['Lower_Envelope']  # Exit when price closes below the lower envelope

        portfolio = vbt.Portfolio.from_signals(
            close=test_data['Close'],
            entries=entries,
            exits=exits,
            init_cash=100_000,
            fees=0.001
        )

        results.append({
            'Year': test_year,
            'Best_Params': best_params
        })

    return pd.DataFrame(results)

# Perform walk-forward optimization with MA Envelope strategy
results = walk_forward_optimization_ma_envelope(df, 2016, 2025)

# Display results
print("\nWalk-Forward Optimization Results:")
print(results)

# Combine signals into a single portfolio
combined_entries = pd.Series(False, index=df.index)
combined_exits = pd.Series(False, index=df.index)

for _, row in results.iterrows():
    year = row['Year']
    params = row['Best_Params']
    yearly_data = df[df.index.year == year]

    # Apply Moving Average Envelope strategy
    yearly_data['Upper_Envelope'], yearly_data['Lower_Envelope'] = calculate_ma_envelope(yearly_data, params[0], params[1])

    # Define entry/exit conditions
    entries = yearly_data['Close'] > yearly_data['Upper_Envelope']
    exits = yearly_data['Close'] < yearly_data['Lower_Envelope']

    combined_entries.loc[entries.index] = entries
    combined_exits.loc[exits.index] = exits

# Filter data for testing period only
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]
combined_entries = combined_entries[(combined_entries.index.year >= 2020) & (combined_entries.index.year <= 2025)]
combined_exits = combined_exits[(combined_exits.index.year >= 2020) & (combined_exits.index.year <= 2025)]

# Backtest using the combined signals
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=combined_entries,
    exits=combined_exits,
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
<ipython-input-41-de222608deec>:70: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-41-de222608deec>:70: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-41-de222608deec>:70: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide


Walk-Forward Optimization Results:
   Year                Best_Params
0  2020  (29, 0.03452865784920211)
1  2021  (22, 0.03698130974989986)
2  2022   (6, 0.04079846187795144)
3  2023   (5, 0.04137231495859229)
4  2024  (5, 0.040629068868308124)
5  2025  (7, 0.028162063182157797)
Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           318709.439756
Total Return [%]                        218.70944
Benchmark Return [%]                   174.090919
Max Gross Exposure [%]                      100.0
Total Fees Paid                       4646.688418
Max Drawdown [%]                         43.31399
Max Drawdown Duration                       734.0
Total Trades                                   18
Total Closed Trades                            17
Total Open Trades                               1
Open Trade PnL     